## Lidar com emails utilizando protocolos POP e SMTP
***

Além das ferramentas que vimos para lidar com troca de arquivos, existem protocolos específicos para lidar com troca de emails: POP e IMAP para receber emails de um servidor e SMTP para mandar emails, e outros formalismos com RFC822 para especificação do conteúdo e formato.

Ferramentas como o Outlook utilizam os protocolos SMTP e POP.

Assim como FTP esses protocolos são montados com base em sockets (portas 110 para POP e 25 para SMTP), e o python também possue uma biblioteca específica para lidar com esses protocolos.

Antes de ver especificamente como mandar e receber emails, vamos abrir o arquivo mailconfig para ver algumas configurações necessárias.

***

In [ ]:
# Servidor de POP3 e seu usuário
servidor_pop = "pop.googlemail.com"
usuario_pop = "victorhad"

In [ ]:
# Servidor de SMTP
# Tem que ver como rodar um servidor smtp na sua máquina (verificar smtpd se quiser subir um servidor smtp)
servidor_smtp = "smtp.gmail.com"
porta_smtp = 587

In [ ]:
# Podemos inserir algumas informações opcionais como signature e endereço
# Além de que pode enviar para o servidor smtp o usuário e senha para poder ser autenticado
usuario_smtp = 'victorhad@gmail.com'

***

Vamos começar com o servidor POP (lêr emails)

***

In [ ]:
import poplib
import getpass
import sys

In [ ]:
# Configurando o servidor
senha = getpass.getpass('Senha para %s: ' % servidor_pop3)

In [ ]:
print('Conectando...')

Por falta de exmplos práticos estaremos conectando no servidor do gmail. Para tanto utilizaremos um objeto especial o POP3_SSL ao inves do objeto clássico do POP3. Esta diferença ocorre porque o servidor do google é implementado por cima de um socket criptografado com SSL. É preciso notar que a porta para servidores desse tipo é, por default, 995, ao invés de 110 do POP3. A porta pode ser configurado especialmente para cada servidor passando o parâmetro como uma string para a construção do objeto.

In [ ]:
servidor = poplib.POP3_SSL(servidor_pop)

In [ ]:
servidor.user(usuario_pop)

In [ ]:
servidor.pass_(senha)

In [ ]:
# Uma vez conectado iniciamos o processo de recebimento de dados
try:
    # Se o servidor tiver alguma mensagem de boas vindas nós o recebemos a partir do método 'getwelcome'
    print(servidor.getwelcome())
    
    # Podemos obter dados da caixa de email utilizando o método stat
    qtd_mensagens, qtd_bytes = servidor.stat()
    
    # Imprimimos as informações recebidas
    print('Existem ', qtd_mensagens, 'mensagens de email em', qtd_bytes, 'bytes')
    
    # Aqui imprimimos as listas de mensagens com os bytes correspondentes a cada uma delas
    print(servidor.list()) # [index_da_mensagem qtd_bytes, ...]
    
    # Esperamos o usuário querer prosseguir para ler o conteúdo das mensagens
    print('-' * 80)
    input('[Pressione Enter para Prosseguir]')
    
    # Iremos percorrer cada uma das mensagens contidas na caixa de emails
    for i in range(qtd_mensagens):
        # Podemos ler o conteúdo das mensagens utilizando o método
        # retr e passando o número da mensagem que nós desejamos obter
        headers, conteudo, octets = servidor.retr(i+1)
        
        # Imprimimos todo o conteúdo da mensagem que acabamos de ler
        # só que temos que decofica-la, uma vez que todo o texto de
        # email está em bytes para o python3.x
        for linha in conteudo: print(linha.decode())
            
        # Imprimimos um separador de conteúdo
        print('-' * 80)
        
        # Se nós não tivermos lido todas as mensagens ainda podemos
        # esperar o usuário pressionar enter para prosseguir com a
        # leitura de mensagens
        if i < qtd_mensagens - 1:
            input('[Pressione Enter para Prosseguir]')
finally:
    # Nós fechamos a conexão com o email a partir do método quit
    servidor.quit()

***

Vamos enviar emails com o SMTP

***

In [ ]:
import smtplib
import email.utils
import getpass

In [ ]:
# Vamos obter a senha de acesso para o servidor SMTP
senha = getpass.getpass('Senha para %s: ' % servidor_smtp)

In [ ]:
# Pede pelas opções de destinatário e remetente.
# Note que podemos ter vários destinatários basta
# separa-los por ;
de = input('De? ').strip()
para = input('Para? ').strip()
paras = para.split(';')

In [ ]:
# Configuramos o assunto da mensagem que informamos a data em formato de rfc2822
assunto = input('Assunto? ').strip()
data = email.utils.formatdate()

In [ ]:
# Criamos os headers de emails padrão
conteudo = ("From: %s\nTo: %s\nDate: %s\nSubject: %s\n\n" % (de, para, data, assunto))

In [ ]:
print(conteudo)

In [ ]:
# Começamos a escrever o conteúdo da mensagem
print('Escreva a mensagem, para finalizar aperte Enter com a mensagem em branco')
while True:
    # Recebemos uma linha digitada pelo usuário
    linha = input('> ')
    
    # Se o usuário terminar de preencher o texto e fornecer uma linha vazia
    # podemos encerrar a mensagem
    if not linha: break
        
    linha += '\n'
    conteudo += linha

In [ ]:
print(conteudo)

In [ ]:
# Iniciamos a conexão com o servidor SMTP
print("Conectando...")
servidor = smtplib.SMTP(servidor_smtp, porta_smtp)

Novamente, para efeitos didáticos estaremos conectando com o gmail. O servidor SMTP do gmail pode ter configuração de um protocolo de criptografia por cima do tipo TLS ou SSL. No caso utilizamos a porta 587, que configura um servidor TLS (poderiamos usar a porta 465 para SSL), e, para servidores deste tipo, é preciso chamar o método 'starttls' antes de começar as operações

In [ ]:
servidor.starttls() # Só usa isso se usar a porta 587

In [ ]:
# Além disso, como se trata do servidor do google, é preciso logar no mesmo
servidor.login(usuario_smtp, senha)

In [ ]:
# Tentamos mandar o email conforme as configurações fornecidas pelo usuário
falhou = servidor.sendmail(de, paras, conteudo)

In [ ]:
# Encerramos a conexão com o servidor smtp
servidor.quit()

In [ ]:
# Se o envio do email tiver falhado nós imprimimos uma mensagem informando os problemas encontrados
if falhou:
    print("Envio falhou:", falhou)
else:
    print("Email enviado com sucesso!")